### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Duchenne_Muscular_Dystrophy/GSE13608'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Adjust based on actual dataset inspection

# Mapping available variables to their respective keys
trait_row = 1 if 'Duchenne Muscular Dystrophy patient' in {x for x in clinical_data[1]} else None
age_row = 2 if any('age' in x for x in clinical_data[2]) else None
gender_row = 3 if any('Gender' in x for x in clinical_data[3]) else None

# Data type conversions
def convert_trait(value):
    if 'Duchenne Muscular Dystrophy patient' in value:
        return 1
    return 0

def convert_age(value):
    try:
        age_str = value.split()[-1]
        return int(age_str)
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    if 'Gender: F' in value:
        return 0
    elif 'Gender: M' in value:
        return 1
    return None

# Save metadata
save_cohort_info('GSE13608', './preprocessed/Duchenne_Muscular_Dystrophy/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if clinical data is available
if trait_row is not None:
    try:
        selected_clinical_data = geo_select_clinical_features(clinical_data, 'Duchenne_Muscular_Dystrophy', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
        csv_path = './preprocessed/Duchenne_Muscular_Dystrophy/trait_data/GSE13608.csv'
        selected_clinical_data.to_csv(csv_path)
        print(preview_df(selected_clinical_data))
    except Exception as e:
        print(f"Error during clinical feature extraction: {e}")
